# Optimizar punto de corte, usando semillero para crear multiples learboard Publicos/Privados

In [109]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [14]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
dataset_file = 'Competencia_01_tagg.csv' # N_cliente, clase_ternaria para junio
dataset_file_shap = 'ensamble_shpa_200_semillas.csv' #N_cliente, Probabiliadd
dataset_file_gv = 'ensamble_gustavo_200_semillas.csv' #N_cliente, Probabiliadd
dataset_file_ensamble='ensamble_shap_gustavo.csv'


clientes_baja_junio= pd.read_csv(dataset_path + dataset_file)
pred_shap = pd.read_csv(dataset_path + dataset_file_shap)
pred_gv = pd.read_csv(dataset_path + dataset_file_gv)
ensamble= pd.read_csv(dataset_path + dataset_file_gv)

In [83]:
ensamble_promedio = pd.concat([pred_shap, pred_gv]).groupby('numero_de_cliente').agg({'Probabilidad': 'mean'}).reset_index()

In [85]:
ensamble_promedio = ensamble_promedio.sort_values(by='Probabilidad', ascending=False).reset_index(drop=True)

In [86]:
ensamble_promedio

,numero_de_cliente,Probabilidad
0,1462097870,0.913762
1,606371992,0.912421
2,627202522,0.899140
3,812656770,0.897985
4,560453687,0.892713
...,...,...
164871,591228473,0.000066
164872,554981460,0.000065
164873,490871275,0.000060
164874,577768545,0.000060


In [3]:
merge=pd.merge(pred_shap,clientes_baja_junio,on="numero_de_cliente", how="inner")

In [4]:
merge[merge["Probabilidad"]>0.08].head(50)

,numero_de_cliente,Probabilidad,clase_ternaria
0,812656770,0.903539,BAJA+1
1,511863915,0.899408,BAJA+1
2,1462097870,0.890252,BAJA+1
3,606371992,0.887768,BAJA+2
4,1021275210,0.867305,BAJA+1
5,303620250,0.865101,CONTINUA
6,560453687,0.865034,CONTINUA
7,1455714205,0.862143,BAJA+2
8,627202522,0.858963,BAJA+1
9,1311956294,0.858812,CONTINUA


In [30]:
merge.iloc[11000:,]

,numero_de_cliente,Probabilidad,clase_ternaria
11000,1134563683,0.068076,BAJA+1
11001,289871747,0.068070,CONTINUA
11002,548160808,0.068067,CONTINUA
11003,1266370748,0.068066,CONTINUA
11004,489876482,0.068054,CONTINUA
...,...,...,...
164871,339394033,0.000105,CONTINUA
164872,554981460,0.000104,CONTINUA
164873,674618747,0.000104,CONTINUA
164874,490871275,0.000090,CONTINUA


In [103]:
def calculoGanancia(bajas,prediccion,corte):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=34477
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

In [104]:
calculoGanancia(clientes_baja_junio,pred_gv,12000)

(106073333.33333334, 86540000.0)

In [107]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)
for corte in cortes:
    ganancia_publico, ganancia_privado = calculoGanancia(clientes_baja_junio, pred_shap, corte)
    
    resultados.append({
    'Corte': corte,
    'Ganancia Público': ganancia_publico,
    'Ganancia Privado': ganancia_privado
})
resultados = pd.DataFrame(resultados)


fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 2.04 segundos


In [108]:
resultados

,Corte,Ganancia Público,Ganancia Privado
0,5000,8.439667e+07,70630000.0
1,6000,8.803667e+07,77470000.0
2,7000,9.501333e+07,84880000.0
3,8000,9.716000e+07,87160000.0
4,9000,9.629667e+07,89930000.0
5,10000,9.902667e+07,91560000.0
6,11000,9.870000e+07,91700000.0
7,12000,9.898000e+07,91980000.0
8,13000,1.010100e+08,91910000.0
9,14000,1.005433e+08,91710000.0


## Tengo el problema que segun la semilla con la que dividi el publico privado me puede dar distinto.. hago semillero

In [90]:
def calculoGanancia(bajas, prediccion, corte, random_state):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia normalizada para el conjunto público.
    ganancia_privado: Ganancia normalizada para el conjunto privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=random_state
    )

    # Obtener los estímulos según el modelo
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # Calcular la ganancia para cada conjunto con normalización
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

def calculoGananciaPromedio(bajas, prediccion, corte, n_seeds=100):
    '''
    Calcula la ganancia promedio para múltiples semillas.
    
    Parámetros:
    bajas: Df con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: Df con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    n_seeds: int, número de semillas aleatorias (por defecto 200).
    
    Retorna:
    promedio_ganancia_publico: Ganancia promedio normalizada para el conjunto público.
    promedio_ganancia_privado: Ganancia promedio normalizada para el conjunto privado.
    '''
    
    seeds = np.random.randint(1000, 100001, size=n_seeds)
    ganancias_publico = []
    ganancias_privado = []

    for seed in seeds:
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, prediccion, corte, random_state=seed)
        ganancias_publico.append(ganancia_publico)
        ganancias_privado.append(ganancia_privado)

    promedio_ganancia_publico = np.mean(ganancias_publico)
    promedio_ganancia_privado = np.mean(ganancias_privado)

    return promedio_ganancia_publico,promedio_ganancia_privado


# prediccion gv

In [65]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)
for corte in cortes:
    promedio_ganancia_publico, promedio_ganancia_privado = calculoGananciaPromedio(clientes_baja_junio, pred_gv, corte, n_seeds=100)
    
    resultados.append({
    'Corte': corte,
    'Ganancia Promedio Público': promedio_ganancia_publico,
    'Ganancia Promedio Privado': promedio_ganancia_privado
})
resultados_gv = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")


Tiempo de ejecución: 191.54 segundos


In [67]:
resultados_gv

,Corte,Ganancia Promedio Público,Ganancia Promedio Privado
0,5000,7.295400e+07,71934000.0
1,6000,8.139693e+07,82715600.0
2,7000,8.676477e+07,87215100.0
3,8000,8.994043e+07,90654100.0
4,9000,9.048293e+07,91621600.0
5,10000,9.171890e+07,92291900.0
6,11000,9.303770e+07,92126700.0
7,12000,9.151753e+07,92778200.0
8,13000,9.323160e+07,92443600.0
9,14000,9.045493e+07,91233600.0


# Prediccion shap

In [59]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)
for corte in cortes:
    promedio_ganancia_publico, promedio_ganancia_privado = calculoGananciaPromedio(clientes_baja_junio, pred_shap, corte, n_seeds=200)
    
    resultados.append({
    'Corte': corte,
    'Ganancia Promedio Público': promedio_ganancia_publico,
    'Ganancia Promedio Privado': promedio_ganancia_privado
})
resultados_shap = pd.DataFrame(resultados)


fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 401.72 segundos


In [60]:
resultados_shap

,Corte,Ganancia Promedio Público,Ganancia Promedio Privado
0,5000,7.457030e+07,74841300.0
1,6000,8.026807e+07,80799400.0
2,7000,8.676955e+07,88413050.0
3,8000,8.990975e+07,90267250.0
4,9000,9.147530e+07,91996300.0
5,10000,9.308145e+07,94107950.0
6,11000,9.321760e+07,94049600.0
7,12000,9.358872e+07,94290550.0
8,13000,9.407510e+07,94882100.0
9,14000,9.399903e+07,94514700.0


# ensamble shap_gv

In [61]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)
for corte in cortes:
    promedio_ganancia_publico, promedio_ganancia_privado = calculoGananciaPromedio(clientes_baja_junio, ensamble, corte, n_seeds=200)
    
    resultados.append({
    'Corte': corte,
    'Ganancia Promedio Público': promedio_ganancia_publico,
    'Ganancia Promedio Privado': promedio_ganancia_privado
})
resultados_ensamble = pd.DataFrame(resultados)


fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 406.93 segundos


In [62]:
resultados_ensamble

,Corte,Ganancia Promedio Público,Ganancia Promedio Privado
0,5000,7.283617e+07,71984500.0
1,6000,8.244122e+07,82268050.0
2,7000,8.690197e+07,87156300.0
3,8000,8.969613e+07,90758800.0
4,9000,9.081380e+07,91479800.0
5,10000,9.190930e+07,92210300.0
6,11000,9.226012e+07,92459950.0
7,12000,9.320173e+07,92056400.0
8,13000,9.260312e+07,92712950.0
9,14000,9.090818e+07,91039350.0


# ensamble shap_g_ promedio

In [91]:
inicio = time.time()

resultados = []
cortes = range(, 20000, 1000)
for corte in cortes:
    promedio_ganancia_publico, promedio_ganancia_privado = calculoGananciaPromedio(clientes_baja_junio, ensamble_promedio, corte, n_seeds=100)
    
    resultados.append({
    'Corte': corte,
    'Ganancia Promedio Público': promedio_ganancia_publico,
    'Ganancia Promedio Privado': promedio_ganancia_privado
})
resultados_ensamble_mean = pd.DataFrame(resultados)


fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 195.32 segundos


In [92]:
resultados_ensamble_mean

,Corte,Ganancia Promedio Público,Ganancia Promedio Privado
0,5000,7.543550e+07,74870500.0
1,6000,8.269567e+07,82159000.0
2,7000,8.860530e+07,88026300.0
3,8000,9.114467e+07,91738000.0
4,9000,9.358557e+07,92691900.0
5,10000,9.500960e+07,93681600.0
6,11000,9.705803e+07,96803700.0
7,12000,9.413740e+07,93255400.0
8,13000,9.210670e+07,92525700.0
9,14000,9.154250e+07,93567500.0
